# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## <span style='color:#2656a3'> 🗒️ The notebook is divided into the following sections:
1. Loading data and process features
2. Connecting to Hopsworks Feature Store
3. Creating feature groups and uploading them to the feature store

## <span style='color:#2656a3'> ⚙️ Import of Libraries and Packages

First, we install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we proceed to import all the necessary libraries.

In [1]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [2]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices and weather measures
from features import electricity_prices, weather_measures 

# We go back into the notebooks folder
%cd notebooks

/Users/camillahannesbo/Documents/AAU/Master - BDS/2. semester/Data Engineering and Machine learning operations in Business/MLOPs-Assignment-
/Users/camillahannesbo/Documents/AAU/Master - BDS/2. semester/Data Engineering and Machine learning operations in Business/MLOPs-Assignment-/notebooks


In [3]:
# Importing the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests
from datetime import datetime, timedelta

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

## <span style="color:#2656a3;"> 💽 Load the Historical Data

The data used comes from different sources:

- Electricity prices in Denmark on hourly basis per day from [Energinet](https://www.energidataservice.dk). Loacated in the *featuresfolder* under electricity_prices.
- Different meteorological observations based on Aalborg Denmark from [Open Meteo](https://www.open-meteo.com). Loacated in the *featuresfolder* under weather_measures.
- Danish calendar that categorizes dates into types based on whether it is a weekday or not. This files is made manually by the group and is located in the *datafolder* inside this repository.
- Forecast Renewable Energy next day from [Energinet](https://www.energidataservice.dk). Loacated in the *featuresfolder* under electricity_prices.
- Weather Forecast based on Aalborg Denmark from [Open Meteo](https://www.open-meteo.com). Loacated in the *featuresfolder* under weather_measures. (This data is used later to parse in new real-time weather data)


### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
This first dataset is Electricity prices on hourly basis per day from Energinet/Dataservice.

In [4]:
# Fetching historical electricity prices for area DK1 from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
electricity_df = electricity_prices.electricity_prices(
    historical=True, 
    area=["DK1"], 
    start='2022-01-01', 
    #end='2023-12-31'
)

In [5]:
# Display the first 5 rows of the electricity dataframe
electricity_df.head(5)

,timestamp,time,date,dk1_spotpricedkk_kwh
0,1640995200000,2022-01-01 00:00:00,2022-01-01,0.37220
1,1640998800000,2022-01-01 01:00:00,2022-01-01,0.30735
2,1641002400000,2022-01-01 02:00:00,2022-01-01,0.32141
3,1641006000000,2022-01-01 03:00:00,2022-01-01,0.33806
4,1641009600000,2022-01-01 04:00:00,2022-01-01,0.28013


In [6]:
# Display the last 5 rows of the electricity dataframe
electricity_df.tail(5)

,timestamp,time,date,dk1_spotpricedkk_kwh
20416,1714503600000,2024-04-30 19:00:00,2024-04-30,0.48640
20417,1714507200000,2024-04-30 20:00:00,2024-04-30,0.48275
20418,1714510800000,2024-04-30 21:00:00,2024-04-30,0.49259
20419,1714514400000,2024-04-30 22:00:00,2024-04-30,0.40340
20420,1714518000000,2024-04-30 23:00:00,2024-04-30,0.38438


In [7]:
# Showing the information for the electricity dataframe
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20421 entries, 0 to 20420
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20421 non-null  int64         
 1   time                  20421 non-null  datetime64[ns]
 2   date                  20421 non-null  object        
 3   dk1_spotpricedkk_kwh  20421 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 638.3+ KB


### <span style="color:#2656a3;">☀️💨 Forecast Renewable Energy next day from Energinet
Second dataset is Forecast Renewable Energy next day also on hourly basis from Energinet

In [8]:
# Fetching historical forecast of renewable energy data for area DK1 from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
forecast_renewable_energy_df = electricity_prices.forecast_renewable_energy(
    historical=True, 
    area = ["DK1"],
    start= '2022-01-01', 
    #end='2023-12-31'
)

In [9]:
# Display the first 5 rows of the forecast_renewable_energy dataframe
forecast_renewable_energy_df.head(5)

,timestamp,time,date,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastintraday_kwh
0,1641024000000,2022-01-01 08:00:00,2022-01-01,0.611708,0.236792,0.000050
1,1641027600000,2022-01-01 09:00:00,2022-01-01,0.459708,0.196667,0.004841
2,1641031200000,2022-01-01 10:00:00,2022-01-01,0.310375,0.178500,0.020353
3,1641034800000,2022-01-01 11:00:00,2022-01-01,0.320750,0.201125,0.035719
4,1641038400000,2022-01-01 12:00:00,2022-01-01,0.355667,0.277667,0.038027


In [10]:
# Display the last 5 rows of the forecast_renewable_energy dataframe
forecast_renewable_energy_df.tail(5)

,timestamp,time,date,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastintraday_kwh
14402,1714503600000,2024-04-30 19:00:00,2024-04-30,0.655292,0.998375,0.163898
14403,1714507200000,2024-04-30 20:00:00,2024-04-30,0.674583,1.042500,0.028304
14404,1714510800000,2024-04-30 21:00:00,2024-04-30,0.707333,1.188708,0.001001
14405,1714514400000,2024-04-30 22:00:00,2024-04-30,0.728000,1.326250,0.000000
14406,1714518000000,2024-04-30 23:00:00,2024-04-30,0.727333,1.366417,0.000000


In [11]:
# Showing the information for the forecast_renewable_energy dataframe
forecast_renewable_energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14407 entries, 0 to 14406
Data columns (total 6 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   timestamp                               14407 non-null  int64         
 1   time                                    14407 non-null  datetime64[ns]
 2   date                                    14407 non-null  object        
 3   dk1_offshore_wind_forecastintraday_kwh  14391 non-null  float64       
 4   dk1_onshore_wind_forecastintraday_kwh   14391 non-null  float64       
 5   dk1_solar_forecastintraday_kwh          14391 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 675.5+ KB


### <span style="color:#2656a3;"> 🌤 Weather measurements from Open Meteo
Next weather measurements from Open Meteo is fetched.

#### <span style="color:#2656a3;"> 🕰️ Historical Weather Measures

In [12]:
# Fetching historical weather measurements from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
historical_weather_df = weather_measures.historical_weather_measures(
    historical=True, 
    start = '2022-01-01', 
    #end = '2023-12-31'
)

In [13]:
# Display the first 5 rows of the weather dataframe
historical_weather_df.head(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1640995200000,2022-01-01,2022-01-01 00:00:00,6.7,100.0,0.0,0.0,0.0,3.0,100.0,16.2,36.0
1,1640998800000,2022-01-01,2022-01-01 01:00:00,6.6,100.0,0.0,0.0,0.0,3.0,100.0,16.2,30.2
2,1641002400000,2022-01-01,2022-01-01 02:00:00,6.7,99.0,0.0,0.0,0.0,3.0,100.0,15.5,30.6
3,1641006000000,2022-01-01,2022-01-01 03:00:00,6.7,100.0,0.0,0.0,0.0,3.0,100.0,12.7,28.8
4,1641009600000,2022-01-01,2022-01-01 04:00:00,6.7,99.0,0.0,0.0,0.0,3.0,100.0,10.6,23.8


In [14]:
# Display the last 5 rows of the weather dataframe
historical_weather_df.tail(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
20395,1714417200000,2024-04-29,2024-04-29 19:00:00,10.6,76.0,0.0,0.0,0.0,0.0,1.0,12.9,21.2
20396,1714420800000,2024-04-29,2024-04-29 20:00:00,9.7,80.0,0.0,0.0,0.0,0.0,0.0,13.4,23.8
20397,1714424400000,2024-04-29,2024-04-29 21:00:00,9.0,82.0,0.0,0.0,0.0,0.0,0.0,11.3,22.7
20398,1714428000000,2024-04-29,2024-04-29 22:00:00,8.6,83.0,0.0,0.0,0.0,0.0,0.0,11.2,19.1
20399,1714431600000,2024-04-29,2024-04-29 23:00:00,8.1,86.0,0.0,0.0,0.0,0.0,0.0,12.0,20.2


In [15]:
# Showing the information for the weather dataframe
historical_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20400 entries, 0 to 20399
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20400 non-null  int64         
 1   date                  20400 non-null  object        
 2   time                  20400 non-null  datetime64[ns]
 3   temperature_2m        20400 non-null  float64       
 4   relative_humidity_2m  20400 non-null  float64       
 5   precipitation         20400 non-null  float64       
 6   rain                  20400 non-null  float64       
 7   snowfall              20400 non-null  float64       
 8   weather_code          20400 non-null  float64       
 9   cloud_cover           20400 non-null  float64       
 10  wind_speed_10m        20400 non-null  float64       
 11  wind_gusts_10m        20400 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 2.0+ M

#### <span style="color:#2656a3;"> 🌈 Weather Forecast
Weather Forecast from Open Meteo is now being fetched. This data is used in part 02 the feature_pipeline to parse in new real-time weather data.

In [16]:
# Fetching weather forecast measures for the next 5 days
weather_forecast_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [17]:
# Display the first 5 rows of the weather_forecast dataframe
weather_forecast_df.head(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1714521600000,2024-05-01,2024-05-01 00:00:00,13.4,70,0.0,0.0,0.0,1,46,20.9,36.4
1,1714525200000,2024-05-01,2024-05-01 01:00:00,12.6,73,0.0,0.0,0.0,0,18,18.0,35.6
2,1714528800000,2024-05-01,2024-05-01 02:00:00,12.0,75,0.0,0.0,0.0,2,54,18.0,31.0
3,1714532400000,2024-05-01,2024-05-01 03:00:00,11.5,76,0.0,0.0,0.0,3,97,19.4,33.1
4,1714536000000,2024-05-01,2024-05-01 04:00:00,11.2,78,0.0,0.0,0.0,3,96,18.0,33.5


In [18]:
# Display the last 5 rows of the weather_forecast dataframe
weather_forecast_df.tail(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
115,1714935600000,2024-05-05,2024-05-05 19:00:00,13.2,81,0.0,0.0,0.0,3,100,6.1,10.8
116,1714939200000,2024-05-05,2024-05-05 20:00:00,12.3,84,0.0,0.0,0.0,3,100,4.0,8.3
117,1714942800000,2024-05-05,2024-05-05 21:00:00,11.6,86,0.0,0.0,0.0,3,100,3.1,5.8
118,1714946400000,2024-05-05,2024-05-05 22:00:00,11.0,88,0.0,0.0,0.0,3,97,3.1,6.1
119,1714950000000,2024-05-05,2024-05-05 23:00:00,10.6,89,0.0,0.0,0.0,3,94,3.5,6.1


In [19]:
# Showing the information for the weather weather_forecast dataframe
weather_forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             120 non-null    int64         
 1   date                  120 non-null    object        
 2   time                  120 non-null    datetime64[ns]
 3   temperature_2m        120 non-null    float64       
 4   relative_humidity_2m  120 non-null    int64         
 5   precipitation         120 non-null    float64       
 6   rain                  120 non-null    float64       
 7   snowfall              120 non-null    float64       
 8   weather_code          120 non-null    int64         
 9   cloud_cover           120 non-null    int64         
 10  wind_speed_10m        120 non-null    float64       
 11  wind_gusts_10m        120 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 11.4+ KB


### <span style="color:#2656a3;"> 🗓️ Calendar of Danish workdays and holidays 
Lastly the calender data is being loaded in. The calendar data includes a 'type' attribute indicating whether the date is a holiday or not

In [20]:
# Read csv file with calender
calender_df = pd.read_csv('https://raw.githubusercontent.com/Camillahannesbo/MLOPs-Assignment-/main/data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])
 
# Display the DataFrame
calender_df.head()

,date,type
0,01/01/2022,Not a Workday
1,02/01/2022,Not a Workday
2,03/01/2022,Workday
3,04/01/2022,Workday
4,05/01/2022,Workday


In [21]:
# Formatting the date column to 'YYYY-MM-DD' dateformat
calender_df["date"] = calender_df["date"].map(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime("%Y-%m-%d"))

In [22]:
# Display the first 5 rows of the calender dataframe
calender_df.head(5)

,date,type
0,2022-01-01,Not a Workday
1,2022-01-02,Not a Workday
2,2022-01-03,Workday
3,2022-01-04,Workday
4,2022-01-05,Workday


In [23]:
# Display the last 5 rows of the calender dataframe
calender_df.tail(5)

,date,type
1091,2024-12-27,Workday
1092,2024-12-28,Not a Workday
1093,2024-12-29,Not a Workday
1094,2024-12-30,Workday
1095,2024-12-31,Workday


In [24]:
# Showing the information for the calender dataframe
calender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1096 non-null   object
 1   type    1096 non-null   object
dtypes: object(2)
memory usage: 17.3+ KB


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

We connect to Hopsworks Feature Store so we can access and create feature groups.

In [25]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550040
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#2656a3;"> 🪄 Creating Feature Groups
A feature group can be seen as a collection of conceptually related features. In this case we create feature groups for the 
- eletricity price data,
- weather data,
- calender data, and
- forecast_renewable_energy.

We specify a `primary_key` as `date`, so we are able to join them when we create a dataset for training later in part 03 the training_pipeline.
We define a name and a short describtion of the feature group's contents and a version number. 

`event_time` is specifyed as `timestamp`.

We've set `online_enabled` to `True` to enable accessing the feature group through the Online API for a Feature View.

In [26]:
# Creating the feature group for the weather data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date","timestamp"], # ,"temperature_2m","relative_humidity_2m","precipitation","rain","snowfall","weather_code","cloud_cover","wind_speed_10m","wind_gusts_10m"
    event_time="timestamp",
    online_enabled=True,
)

We have now outlined metadata for the feature group. Data hasn't been stored yet, and there's no schema defined. To store data persistently for the feature group, we populate it with its associated data using the `insert` function.

In [27]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(historical_weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550040/fs/545863/fg/768380


Uploading Dataframe: 100.00% |██████████| Rows 20400/20400 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x13bbc84d0>, None)

We make a descriptions for each feature we put into the feature group. In this way we are adding more information and documentation to the user.

In [28]:
# List of descriptions for weather features
weather_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the weather measurement"},
    {"name": "time", "description": "Time of the weather measurement"},
    {"name": "temperature_2m", "description": "Temperature at 2m above ground"},
    {"name": "relative_humidity_2m", "description": "Relative humidity at 2m above ground"},
    {"name": "precipitation", "description": "Precipitation"},
    {"name": "rain", "description": "Rain"},
    {"name": "snowfall", "description": "Snowfall"},   
    {"name": "weather_code", "description": "Weather code"},   
    {"name": "cloud_cover", "description": "Cloud cover"},   
    {"name": "wind_speed_10m", "description": "Wind speed at 10m above ground"},   
    {"name": "wind_gusts_10m", "description": "Wind gusts at 10m above ground"},   
]

# Updating feature descriptions
for desc in weather_feature_descriptions: 
    weather_fg.update_feature_description(desc["name"], desc["description"])

We replicate the process for both the `electricity_fg`, `forecast_renewable_energy_fg` and `danish_holidays_fg` by establishing feature groups and inserting the dataframes into their respective feature groups.

In [29]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date","timestamp"], # "dk1_spotpricedkk_kwh"
    online_enabled=True,
    event_time="timestamp",
)

In [30]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550040/fs/545863/fg/770401


Uploading Dataframe: 100.00% |██████████| Rows 20421/20421 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/electricity_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x13bb23090>, None)

In [31]:
# List of descriptions for electricity features
electricity_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the electricity measurement"},
    {"name": "time", "description": "Time of the electricity measurement"},
    {"name": "dk1_spotpricedkk_kwh", "description": "Spot price in DKK per KWH"}, 
]

# Updating feature descriptions
for desc in electricity_feature_descriptions: 
    electricity_fg.update_feature_description(desc["name"], desc["description"])

In [32]:
# Creating the feature group for the electricity prices
forecast_renewable_energy_fg = fs.get_or_create_feature_group(
    name="forecast_renewable_energy",
    version=1,
    description="Forecast on Renewable Energy on ForecastType from Energidata API",
    primary_key=["date","timestamp"], # ,"dk1_offshore_wind_forecastintraday_kwh","dk1_onshore_wind_forecastintraday_kwh","dk1_solar_power_forecastintraday_kwh"
    online_enabled=True,
    event_time="timestamp",
)

In [33]:
# Inserting the electricity_df into the feature group named electricity_fg
forecast_renewable_energy_fg.insert(forecast_renewable_energy_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550040/fs/545863/fg/770402


Uploading Dataframe: 100.00% |██████████| Rows 14407/14407 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: forecast_renewable_energy_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/forecast_renewable_energy_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x13bbfdb50>, None)

In [34]:
# List of descriptions for forecast_renewable_energy features
forecast_renewable_energy_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date"},
    {"name": "time", "description": "Time for the event_time"},
    {"name": "dk1_offshore_wind_forecastintraday_kwh", "description": "The forecast for the coming day at 6am Danish time zone"},
]

# Updating feature descriptions
for desc in forecast_renewable_energy_feature_descriptions: 
    forecast_renewable_energy_fg.update_feature_description(desc["name"], desc["description"])

In [35]:
# Creating the feature group for the danish holidays
danish_holidays_fg = fs.get_or_create_feature_group(
    name="danish_holidayss",
    version=1,
    description="Danish holidays calendar.",
    online_enabled=True,
    primary_key=["date"],
)

In [36]:
# Inserting the calender_df into the feature group named danish_holidays_fg
danish_holidays_fg.insert(calender_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550040/fs/545863/fg/768382


Uploading Dataframe: 100.00% |██████████| Rows 1096/1096 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: danish_holidayss_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/danish_holidayss_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x13bcc6a90>, None)

In [37]:
# List of descriptions for danish_holidays features
danish_holidays_feature_descriptions = [
    {"name": "date", "description": "Date in the calendar"},
    {"name": "type", "description": "Holyday or not holyday"},
]

# Updating feature descriptions
for desc in danish_holidays_feature_descriptions: 
    danish_holidays_fg.update_feature_description(desc["name"], desc["description"])

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

Next we will generate new data for the Feature Groups.